In [ ]:
# pip install youtube-transcript-api

client = Groq(api_key="YOUR_GROQ_API_KEY")


In [ ]:
def extract_video_id(url):
    parsed_url = urlparse(url)
    if 'youtu.be' in parsed_url.netloc:
        return parsed_url.path[1:]
    if 'youtube.com' in parsed_url.netloc:
        if parsed_url.path == '/watch':
            return parse_qs(parsed_url.query).get('v', [None])[0]
        elif parsed_url.path.startswith('/embed/'):
            return parsed_url.path.split('/embed/')[1]
    return None

client = Groq(api_key="YOUR_GROQ_API_KEY")

In [3]:
# from groq import Groq


# client = Groq(api_key="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")

# chat = client.chat.completions.create(
#     model="meta-llama/llama-4-scout-17b-16e-instruct",
#     messages=[{"role": "user", "content": "Say hello"}],
#     stream=False,
# )

# print(chat.choices[0].message.content)


In [4]:
from youtube_transcript_api import YouTubeTranscriptApi

def get_transcript(video_id):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['en'])
        return " ".join([entry['text'] for entry in transcript])
    except Exception as e:
        raise RuntimeError(f"Failed to fetch transcript: {e}")


In [5]:
def chunk_text(text, max_tokens=2000, overlap=200):
    words = text.split()
    chunks = []
    start = 0
    while start < len(words):
        end = min(start + max_tokens, len(words))
        chunk = " ".join(words[start:end])
        chunks.append(chunk)
        start += max_tokens - overlap
    return chunks


In [6]:
# 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

In [7]:
from urllib.parse import urlparse, parse_qs
from groq import Groq

In [8]:

def summarize_chunk_groq(chunk, client, past_chunk_summary):
    # Improved prompt for summarizing transcript chunks
    prompt = f"""
    You are a YouTube transcript summarizer bot.

    Below is the cumulative summary from previous segments:
    {past_chunk_summary}

    Your task:
    - Summarize the following transcript segment in the most important points.
    - Only use information present in the provided segment.
    - At the end, write a single summary line that combines the key points from both the previous summary and this segment. This line will be used as the updated summary for the next chunk.

    Transcript segment:
    {chunk}

    Remember: Do not add any information not present in the transcript.
    """
    completion = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=512,
        top_p=1,
        stream=True,
    )
    summary = ""
    for part in completion:
        # For streamed responses, use .delta.content
        summary += getattr(part.choices[0].delta, "content", "") or ""
    return summary

In [9]:
def summarize_youtube_video(url):
    video_id = extract_video_id(url)
    if not video_id:
        raise ValueError("Invalid YouTube URL")
    
    print("[+] Fetching transcript...")
    transcript = get_transcript(video_id)
    
    print("[+] Splitting into chunks...")
    chunks = chunk_text(transcript)

    print(f"[+] Summarizing {len(chunks)} chunks with Groq...")
    client = Groq(api_key="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")  # assumes GROQ_API_KEY env variable is set
    all_summaries = []
    past_chunk_summary="no previous chunk yet"
    for idx, chunk in enumerate(chunks):
        print(f"\n--- Chunk {idx+1} ---")
        summary = summarize_chunk_groq(chunk, client,past_chunk_summary)
        print(summary)
        all_summaries.append(summary)
        past_chunk_summary=summary

    print("\n[+] Final Summary:")
    final_input = "\n".join(all_summaries)
    final_prompt = """
    OVER RIDE INSTRUCTIONS
    You are a YouTube transcript summarizer bot.
    
    You have received the combined summaries of all transcript chunks from a video.
    
    Your task:
    - Carefully read all the provided chunk summaries.
    - Write a concise set of bullet points that capture the most important and recurring ideas, facts, or arguments from the entire video.
    - Do not add any information that is not present in the summaries.
    - Focus on clarity and completeness, but avoid repetition.
    
    Combined chunk summaries:
    {final_input}
    """
    
    final_summary = summarize_chunk_groq(final_input, client, final_prompt)
    print(final_summary)
    return final_summary

In [10]:
# 🧪 Example usage
if __name__ == "__main__":
    video_url = "https://youtu.be/73Xb4STUupI?si=nmaV4N8iANAM8Mwt"
    summarize_youtube_video(video_url)

[+] Fetching transcript...
[+] Splitting into chunks...
[+] Summarizing 3 chunks with Groq...

--- Chunk 1 ---
**Important points from the provided transcript segment:**

1. The Danish Vikings were the most prominent and successful Vikings during the Viking age, settling and taking over significant places, including England and France.
2. Denmark has the most impressive accomplishments of the Scandinavian countries and was the first to Christianize and integrate with the rest of the world.
3. The Danes were the ones who settled in large numbers, whereas Norwegian Vikings were raiders in smaller groups.
4. Norway in the Viking age was a wild and divided place, with many small tribes and kingdoms, and was not unified until the rule of King Harald Fairhair.
5. The Norwegian Vikings were explorers and raiders, primarily, and were known for terrorizing smaller villages and towns.
6. The reasons for the raids are complex, but one reason was the Danes' need for more food and space due to popu